

Important: in kernel-only competitions we can't use internet connections. So I use pretrained models from here: https://www.kaggle.com/bminixhofer/pytorch-pretrained-image-models

In [1]:
#!pip install "../input/pretrained-models/pretrained-models/pretrained-models.pytorch-master"

In [2]:
import sys
package_path = "../input/efficientnet-pytorch/efficientnet-pytorch/EfficientNet-PyTorch-master"
sys.path.append(package_path)

In [3]:
# libraries
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import time 
import tqdm
from PIL import Image
train_on_gpu = True
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
import copy
import cv2
import albumentations
from albumentations import torch as AT
import random

from efficientnet_pytorch import EfficientNet
#import pretrainedmodels

## Data overview

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [5]:
seed_everything(1234)

In [6]:
train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
sample_submission = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

## Data preparation

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
def prepare_labels(y):
    # From here: https://www.kaggle.com/pestipeti/keras-cnn-starter
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

    y = onehot_encoded
    return y, label_encoder

In [8]:
y, le = prepare_labels(train['diagnosis'])

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [9]:
class GlassDataset(Dataset):
    def __init__(self, df, datatype='train', transform = transforms.Compose([transforms.CenterCrop(32),transforms.ToTensor()]), y = None):
        self.df = df
        self.datatype = datatype
        if self.datatype == 'train':
            self.image_files_list = [f'../input/aptos2019-blindness-detection/{self.datatype}_images/{i}.png' for i in df['id_code'].values]
            #self.image_files_list = [i for i in df['id_code'].values]
            self.labels = y
        else:
            self.image_files_list = [f'../input/aptos2019-blindness-detection/{self.datatype}_images/{i}.png' for i in df['id_code'].values]
            self.labels = np.zeros((df.shape[0], 5))
        self.transform = transform

    def __len__(self):
        return len(self.image_files_list)

    def __getitem__(self, idx):
        img_name = self.image_files_list[idx]
        img = cv2.imread(img_name)
        image = self.transform(image=img)
        image = image['image']

        img_name_short = self.image_files_list[idx].split('.')[0]

        label = self.labels[idx]
        if self.datatype == 'test':
            return image, label, img_name
        else:
            return image, label

In [10]:
data_transforms_test = albumentations.Compose([
    albumentations.Resize(256, 256),
    albumentations.HorizontalFlip(),
    albumentations.ShiftScaleRotate(rotate_limit=15, scale_limit=0.10),
    albumentations.Normalize(),
    AT.ToTensor()
    ])

batch_size = 16
num_workers = 0

test_set = GlassDataset(df=test, datatype='test', transform=data_transforms_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=num_workers)


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [12]:
import os
os.listdir("../input/efffin-end")

['eff4_fin.pt']

In [13]:
"""model_conv1 = EfficientNet.from_name('efficientnet-b0')
in_features = model_conv1._fc.in_features
model_conv1._fc = nn.Linear(in_features, 5)
model_conv1.load_state_dict(torch.load("../input/effnet-new/effnet0_new_2.pt"))
for param in model_conv1.parameters():
    param.requires_grad = False
model_conv1.to(device)"""

'model_conv1 = EfficientNet.from_name(\'efficientnet-b0\')\nin_features = model_conv1._fc.in_features\nmodel_conv1._fc = nn.Linear(in_features, 5)\nmodel_conv1.load_state_dict(torch.load("../input/effnet-new/effnet0_new_2.pt"))\nfor param in model_conv1.parameters():\n    param.requires_grad = False\nmodel_conv1.to(device)'

In [14]:
model_conv2 = EfficientNet.from_name('efficientnet-b4')
in_features = model_conv2._fc.in_features
model_conv2._fc = nn.Linear(in_features, 5)
model_conv2.load_state_dict(torch.load("../input/efffin-end/eff4_fin.pt"))
for param in model_conv2.parameters():
    param.requires_grad = False
model_conv2.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

## Predicting

In [15]:
def test_model(model, test_loader):
    sub = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
    model.eval()
    for (data, target, name) in test_loader:
        data = data.cuda()
        output = model(data)
        output = output.cpu().detach().numpy()
        for i, (e, n) in enumerate(list(zip(output, name))):
            sub.loc[sub['id_code'] == n.split('/')[-1].split('.')[0], 'diagnosis'] = le.inverse_transform([np.argmax(e)])
    print( "done")
    return sub

In [16]:
coef = [0.5, 1.5, 2.5, 3.5]
#coef = [0.57, 1.37, 2.57, 3.57]

In [17]:
test_resnet_21 = test_model(model_conv2, test_loader)
test_resnet_22 = test_model(model_conv2, test_loader)
test_resnet_23 = test_model(model_conv2, test_loader)
test_resnet_24 = test_model(model_conv2, test_loader)
test_resnet_25 = test_model(model_conv2, test_loader)



done
done
done
done
done


In [18]:
test_preds_3 = (test_resnet_21.diagnosis + test_resnet_22.diagnosis + test_resnet_23.diagnosis + test_resnet_24.diagnosis + test_resnet_25.diagnosis  )  / 5

In [19]:
for i, pred in enumerate(test_preds_3):
    if pred < coef[0]:
        test_preds_3[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        test_preds_3[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        test_preds_3[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        test_preds_3[i] = 3
    else:
        test_preds_3[i] = 4

In [20]:
sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = test_preds_3.astype(int)
sample.to_csv("submission.csv", index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 